In [1]:
import numpy as np
import librosa
import google.generativeai as genai
import yaml
import time

In [2]:
og_arr, og_sr = librosa.load("data/music.mp3")
og_arr

array([-1.3862776e-02, -1.7331528e-02, -1.6357925e-02, ...,
        9.8206228e-06, -1.7608858e-07,  9.6336289e-06], dtype=float32)

In [28]:
freq_arr = librosa.stft(og_arr)

### zero shot audio detection
* [clap model](https://huggingface.co/docs/transformers/model_doc/clap): it requires a text input which it yields a probability for. Perhaps get the set of species based on location and then use the model to tell us which one it could be? But the training set would require to have the insect sounds. Dataset of insect noises then to train on/
* just into a multimodal llm?

In [6]:
with open("config.yaml", "r") as conf_file:
    config = yaml.safe_load(conf_file)
genai.configure(api_key=config["GEMINI_KEY"])

In [7]:
def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

In [8]:
audio_file = upload_to_gemini("data/music.mp3", mime_type="audio/x-mp3")

Uploaded file 'music.mp3' as: https://generativelanguage.googleapis.com/v1beta/files/r3mk9g51uz0h


In [16]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")
prompt = "give me the exact drum pattern in this song so I can recreate it in a sequencer"
response = model.generate_content([prompt, audio_file])
print(response.text)

Unfortunately, I cannot listen to and process audio information like identifying the drum pattern from a song. If you could provide a description of the drum pattern like kick on 1 & 3, snare on 2 & 4, that would be helpful. 

However, I can give you some general tips on recreating drum patterns in a sequencer:

1. **Identify the tempo:** Most sequencers allow you to set a specific tempo (BPM) which is crucial for accurate recreation.
2. **Break down the pattern:** Listen carefully for kick, snare, hi-hat, and any other percussive elements. Focus on identifying where they land within each measure. 
3. **Start with the basics:** Begin by laying down the kick and snare pattern as they form the foundation of most drum grooves. 
4. **Layer the hi-hats:** Experiment with different hi-hat patterns to add groove and complexity.
5. **Fine-tune and add variation:** Don't be afraid to adjust the velocity and timing of individual hits to create a more human feel.

Let me know if you have any more